# CS486 - Artificial Intelligence
## Lesson 3 - Informed Search

We can improve tree search with knowledge about the problem domain. For example, for US currency, returning the largest coin that doesn't exceed the goal always will always produce an optimal result. So how can we apply that knowledge to our change problem? 

First, we'll need to import the AIMA search library:

In [1]:
from helpers import counter
from aima.search import *

## Greedy Search

The **`greedy_best_first_graph_search`** function takes a heuristic function that estimates the cost expanding the node. It's *greedy* because it *always* chooses the path with the cheapest cost according to the heuristic. Let's try running a greedy search against our **`Change`** problem from the previous lesson.

First, enter your **`Change`** class from the last time in the cell below:

In [7]:
class Change(Problem):
    def coins(self):
        return (25,10,5,1)
    
    def actions(self,state):
        return (coin for coin in self.coins() if coin + sum(state) <= self.goal)
    
    def result(self,state,action):
        return state + (action,)
    
    @counter
    def goal_test(self, state):
        return sum(state) == self.goal

Now we'll use the **`greedy_best_first_graph_search`** function to search your Change tree for a solution. The search function takes two arguments: The problem instance and a heuristic. The heuristic takes a **`Node`** object that encodes the current node in the tree, the action that lef to the node, and the node's corresponding state. Here's what the **`Node`** class looks like:

In [8]:
%pdoc Node

Class docstring:
    A node in a search tree. Contains a pointer to the parent (the node
    that this is a successor of) and to the actual state for this node. Note
    that if a state is arrived at by two paths, then there are two nodes with
    the same state.  Also includes the action that got us to this state, and
    the total path_cost (also known as g) to reach the node.  Other functions
    may add an f and h value; see best_first_graph_search and astar_search for
    an explanation of how the f and h values are handled. You will not need to
    subclass this class.
Init docstring:
    Create a search tree Node, derived from a parent by an action.

Below is code that implements a heuristic that always returns 1. Edit the **`heuristic`** function so that it always selects nodes with the highest value. 

**Hint:** `node.state` contains the list of coins for the state we're calculating the heuristic for.

In [9]:
def heuristic(node):
    total = sum(node.state)
    return len(node.state) + (1/total if total > 0 else 1/0.0001) 

change = Change(initial=(),goal=51)
greedy_best_first_graph_search(change, heuristic)

<Node (1, 25, 25)>

Using the same code from last the Uniformed Search lesson, we can see how greedy search performs by adding it to the list of searches below.

In [12]:
def greedy(problem):
    return greedy_best_first_graph_search(problem, heuristic)

def astar(problem):
    return astar_search(problem, heuristic)

searches = [
    breadth_first_tree_search,    
    iterative_deepening_search,
    uniform_cost_search,
    depth_first_tree_search,
    greedy,
    astar
]

print("{:^26} {:^10} {:^16}".format("Search Type", "Goal Tests", "Solution"))

for search in searches:
    problem = Change(initial=(),goal=26)
    result = search(change)
    print("{:26} {:^11} {:20}".format(search.__name__,problem.goal_test.count,str(result.solution())))

       Search Type         Goal Tests     Solution    
breadth_first_tree_search      331     [25, 25, 1]         
iterative_deepening_search     362     [25, 25, 1]         
uniform_cost_search            400     [1, 25, 25]         
depth_first_tree_search        452     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
greedy                         475     [1, 25, 25]         
astar                          498     [1, 25, 25]         


## A\* search
Is greedy *always* better? Consider the problem below.

In [16]:
class CarnivalChange(Change):
    def coins(self):
        return [1,3,4]

print("{:^26} {:^10} {:^16}".format("Search Type", "Goal Tests", "Solution"))

for search in searches:
    change = CarnivalChange(initial=(),goal=15)
    result = search(change)
    print("{:26} {:^11} {:20}".format(search.__name__,change.goal_test.count,str(result.solution())))

       Search Type         Goal Tests     Solution    
breadth_first_tree_search      364     [4, 4, 4, 4, 4]     
iterative_deepening_search     543     [4, 4, 4, 4, 4]     
uniform_cost_search            365     [4, 4, 4, 4, 4]     
depth_first_tree_search         6      [4, 4, 4, 4, 4]     
greedy                         123     [4, 4, 4, 4, 4]     
astar                          123     [4, 4, 4, 4, 4]     


Greedy seems to perform well but, in the **`CarnivalChange`** case, isn't returning the optimal solution. So how do we get the performance of a greedy search with the accuracy of Uniform Cost Search? The answer is **A\* search**. 

The A* algorithm takes both a heuristic function and the path cost and adds the two together:

\begin{equation*}
f(n) = g(n) + h(n)
\end{equation*}

A\* is guaranteed to return the optimal path if the heuristic is ***admissible***, meaning that it never over-estimates the cost to the goal. Consider the following questions before continuing:

* Is our heuristic admissible? Why or why not?
* What is the path cost of our **`Change`** problem? Can we change it? Do we need to?

The AIMA **`astar_search`** operates exactly like the **`greedy_best_first_graph_search`** function: It takes a problem instance and a heuristic function. Using the code above as a reference, run A\* against the **`CarnivalChange`** problem and answer the following questions:

* Does A\* find the optimal solution? 
* How does A\* perform compared to the other search algorithms?

In [ ]:
# Run A* against CarnivalChange and evaluate its performance here